
# kNN in Python

## Vorbereitung

### Module laden

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns

from scipy.stats import zscore

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier 

from bdsm import quality
from bdsm.datasets import iris

### Daten laden

In [ ]:
df = iris()

In [ ]:
df.head()

In [ ]:
df = df.to_numeric()
df.head()

In [ ]:
quality(df)

### Prädiktormatrix und Responsevariable definieren

In Python ist es notwendig, die Prädiktormatrix $X$ und die Responsevariable $y$ in **zwei getrennten** Objekten zu speichern. Obwohl kNN eigentlich kein Training durchführt, ist es üblich, auch hier ein Test- und Trainingsset zu definieren. 

Es gibt in `sklearn` die Funktion `train_test_split`, die einen gegebenen Datensatz nach bestimmten Parametern in Trainings- und Testdaten teilt.

In [ ]:
X = df.drop("Class_cat", axis=1)
y = df['Class_cat']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=147)

### Daten visualisieren

In [ ]:
sns.pairplot(df, hue='Class_cat');

## kNN durchführen

kNN wird in 3 Schritten durchgeführt:

1. die Parameter der kNN festlegen (Wert für $k$, Distanzfunktion)
1. das Modell mit den Trainingsdaten anpassen
1. die Distanzen zu den $k$ nächsten Nachbarn für jeden Datenpunkt der Testdaten berechnen und die Zugehörigkeit bestimmen

Sind zwei Punkte gleich weit entfernt, bestimmt die Sortierung der Trainingsdaten welcher genommen wird.

### Ohne Standardisierung

Zunächst betrachten wir ein Modell mit 5 Nachbarn und der euklidischen Distanz. Die Datenpunkte werden nicht standardisiert. Die euklidische Distanz ist eine Minkowski-Metrik mit $p=2$. Daher benötigt unser Klassifikator folgende Parameter: 

- `n_neighbors=5`
- `p=2`
- `metric='minkowski'`

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')

In [ ]:
knn.fit(X_train, y_train)

Ein weiterer wichtiger Parameter, den man variieren kann, ist `weights`. Er akzeptiert folgende Werte:

- `uniform`: Alle Punkte in der Umgebung haben das gleiche Gewicht
- `distance`: Die Punkte werden invers zu ihrem Abstand (1/Abstand) gewichtet. Je näher sich ein Punkt befindet, desto wichtiger ist er für die Klassenzugehörigkeit

Weitere Informationen: 
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
pred = knn.predict(X_test)

Wir wollen nun wissen, wie gut unser Modell klassifiziert hat.

In [ ]:
from bdsm.metrics import confusion_matrix

In [ ]:
conf_matrix = confusion_matrix(pred, y_test)
conf_matrix

Grafikfreunde können sich die Confusion_matrix auch als Heatmap anzeigen lassen:

In [ ]:
sns.heatmap(conf_matrix, annot=True, fmt="0");

Die Accuracy kann man sich einfach über `accuracy_score` ausgeben lassen:

In [ ]:
accuracy_score(y_test, pred)

Eine detailliertere Auswertung bekommt man mit `classification_report`:

In [ ]:
print(classification_report(y_test,pred))

`support` gibt an, wie oft das Merkmal in den Testdaten vorhanden war.

### Mit Standardisierung

In [ ]:
X_train_z = zscore(X_train)
X_test_z = zscore(X_test)

In [ ]:
knn_z = KNeighborsClassifier(n_neighbors=5,p=2,metric='minkowski')

In [ ]:
knn_z.fit(X_train_z, y_train)

In [ ]:
pred_z = knn_z.predict(X_test_z)

In [ ]:
conf_matrix_z = confusion_matrix(pred_z, y_test)
conf_matrix_z

In [ ]:
accuracy_score(y_test, pred_z)

In [ ]:
print(classification_report(y_test,pred_z))

### Mit Cross-Validation

Da uns in diesem einfachen Beispiel die nicht-standardisierten Werte bessere Ergebnisse liefern, machen wir mit diesen weiter. 

Die Ergebnisse der CV speichern wir in einem Array, das wir zunächst erzeugen:

In [ ]:
cv_scores = []

Nun führen wir eine 10-fold CV durch und speichern die mittlere Accuracy in unserem `cv_scores`-Array:

In [ ]:
cv_scores = []
k_values = range(2,50)
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors = k)
    scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

Suchen wir das optimale $k$:

In [ ]:
plt.plot(k_values, cv_scores);

Was ist nun das beste $k$? Erzeugen wir uns einen Pandas-Dataframe und suchen uns die Einträge mit der höchsten Accuracy:

In [ ]:
k_acc = pd.DataFrame({'k': k_values, 'Acc': cv_scores})
k_acc[k_acc['Acc'] == max(k_acc['Acc'])]

Nun könnten wir noch weiter Parameter variieren, um zwischen den beiden resultierenden $k$s zu unterscheiden.